# **Workshop -001: Extract**

**Importación de las librerias a utilizar**

In [1]:
import yaml
import psycopg2 
from psycopg2 import sql
from sqlalchemy import create_engine, text
import pandas as pd
import matplotlib.pyplot as plt

Matplotlib is building the font cache; this may take a moment.


**Creamos una función donde leemos el archivo de configuración de la DB y cargamos los datos de la conexión**

In [2]:
def load_config(file_path="config.yaml"):
    with open(file_path, "r") as file:
        return yaml.safe_load(file)

**Llamamos a la funcion que carga los dados de conexión a la base de datos, y creamo la conexión**

In [3]:
config = load_config()
db_config = config["database"]

db_user = db_config["user"]
db_password = db_config["password"]
db_host = db_config["host"]
db_port = db_config["port"]
db_name = db_config["name"]

conn = psycopg2.connect(
    dbname="postgres",
    user=db_user,
    password=db_password,
    host=db_host,
    port=db_port
)
conn.autocommit = True

**Creamos la base de datos en caso de que no exista**

In [4]:
db_name = "etl_project"
try:
    with conn.cursor() as cur:
        cur.execute(sql.SQL("CREATE DATABASE {}").format(sql.Identifier(db_name)))
        print(f"Base de datos '{db_name}' creada exitosamente.")
except psycopg2.errors.DuplicateDatabase:
    print(f"La base de datos '{db_name}' ya existe.")
finally:
    conn.close()

Base de datos 'etl_project' creada exitosamente.


**Creamos las tablas necesarias a utilizar en el proyecto**

In [5]:
engine = create_engine(f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")

with engine.connect() as conn:
    conn.execute(text("""
        CREATE TABLE saldos_staging (
            id BIGSERIAL PRIMARY KEY,
            documento_identidad VARCHAR(20),
            nombre VARCHAR(100),
            apellido VARCHAR(100),
            sexo CHAR(1),
            estado_civil INT,
            fecha_ingreso date,
            tipo_salario INT,
            salario NUMERIC(15, 2),
            estrato INT,
            tipovehiculo INT,
            lincred INT,
            fecsolic date,
            fecaprob date,
            fecfact date,
            fecdesc date,
            fecultcau date,
            fecultpago date,
            fecvemto date,
            plazo INT,
            vlrsolicitud NUMERIC(15, 2),
            valorob NUMERIC(15, 2),
            saldot NUMERIC(15, 2),
            cuota NUMERIC(15, 2),
            tasaint NUMERIC(3, 2),
            ciclod CHAR(1),
            periodd CHAR(1),
            clacuo CHAR(1),
            clasei CHAR(1),
            clades CHAR(1),
            periodo INT,
            saldo NUMERIC(15, 2),
            saldo_inicial NUMERIC(15, 2),
            vlr_debito NUMERIC(15, 2),
            vlr_credito NUMERIC(15, 2),
            cuopen INT,
            valor_pagado NUMERIC(15, 2),
            fecha_pago VARCHAR(20),
            mora_causado NUMERIC(15, 2),
            mora_abono NUMERIC(15, 2),
            mora_saldo NUMERIC(15, 2),
            descripcion VARCHAR(100),
            codahor CHAR(1),
            debcre CHAR(1),
            fecha_registro TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        );
    """))
    conn.commit() 
    print("Tabla: saldos_staging creada exitosamente en PostgreSQL.")

Tabla: saldos_staging creada exitosamente en PostgreSQL.


**Leemos los datos del archivo CSV, e imprimimos 15 registros**

In [6]:
df = pd.read_csv('saldos.csv', sep=';', low_memory=False, dtype={    
    'vlr_debito': 'float64',
    'vlr_credito': 'float64',
    'cuopen': 'Int64',
    'valor_pagado': 'float64',    
    'mora_causado': 'float64',
    'mora_abono': 'float64',
    'mora_saldo': 'float64'
}, encoding='ISO-8859-1')

In [7]:
print(df.sample(n=15))

        documento_identidad            nombre            apellido sexo  \
11837              13012814      JAIRO MIGUEL         ARANGO RUIZ    M   
123660           1006432766    YEILYN XIOMARA  CARABALI ARREDONDO    F   
121606           1006013367   MELANY GABRIELA       GARCIA RIVERA    F   
150675           1118290581   ANDRES MAURICIO   CARVAJAL GALLARDO    M   
7609                6555413           ALBERTO       OSORIO MOTATO    M   
120298             94529609    JULIAN ALBERTO        VIDAL ZU IGA    M   
133094           1118283418    DIEGO FERNANDO        GARCIA REYES    M   
31947              16456102           SALOMON   QUINTANA MOSQUERA    M   
86045              31478868  GLORIA ERNESTINA             HERRERA    F   
17391              16446943   GABRIEL ANTONIO       PECHENE CHAUX    M   
53362              24718126    SANDRA BIBIANA      USECHE GALINDO    F   
68587              31141888       MARIA NISMA     HERRERA GAVIRIA    F   
171405           1144160223   JHOINNER

**Copiamos los datos en una nueva variable, para guardarlos en la tabla saldos_staging**

In [ ]:
df_staging = df.copy()

df_staging.sample(n=15)

,documento_identidad,nombre,apellido,sexo,estado_civil,fecha_ingreso,tipo_salario,salario,estrato,tipovehiculo,...,vlr_credito,cuopen,valor_pagado,fecha_pago,mora_causado,mora_abono,mora_saldo,descripcion,codahor,debcre
53750,25265861,ANA BELIA,BALLESTEROS VALDES,F,3,28/10/2010,2,806085,0,0,...,3339.0,0,3339.0,31/01/2024,0.0,0.0,0.0,03 INT. VISTA,2,C
130615,1118256309,ALEJANDRO,SANCHEZ MEDINA,M,1,2/03/2016,4,1677500,3,0,...,0.0,7,NaN,NaN,4790.0,0.0,4790.0,SERVICIO EXEQUIAL,3,D
59789,29946049,SORY ALEYDA,GUINCHIN MARIN,F,2,26/02/2018,4,1677500,3,0,...,33550.0,0,33550.0,31/08/2023,33550.0,33550.0,0.0,APORTES,1,C
58047,29938425,LUZ ESTELLA,FRANCO MEDINA,F,4,27/06/2008,4,2065000,0,0,...,401768.0,84,879146.0,28/10/2024,1318719.0,879146.0,439573.0,PTMO CONSUMO,4,D
40267,16468233,WALTER JHON,PAYA CABRERA,M,4,15/03/2011,4,3690151,2,0,...,43656.0,11,43656.0,30/06/2024,43656.0,43656.0,0.0,BON-SOL,3,D
92610,31486708,ANGELA ADRIANA,REYES GUEVARA,F,2,7/02/2018,4,1677500,3,0,...,0.0,0,NaN,NaN,NaN,NaN,NaN,REVALORIZACION,3,C
139380,1118288176,KATHERINE DE LOS ANGELES,PUENTE FERREROSA,F,4,21/03/2023,4,1677500,3,0,...,33619.0,0,33619.0,30/04/2024,33550.0,33550.0,0.0,A/PERM,2,C
140258,1118284992,ANGELICA LILIANA,LOZADA AYALA,F,1,19/04/2023,4,3927000,2,0,...,27781.0,3,27781.0,30/09/2024,27781.0,27781.0,0.0,BON-SOL,3,D
150570,1118295693,ALISSON YULIANA,RUIZ SERNA,F,2,15/04/2016,4,3927000,2,0,...,0.0,10,NaN,NaN,536157.0,0.0,536157.0,PTMO CONSUMO,4,D
126032,1094888660,DANIELA,MOLANO MEDINA,F,1,18/09/2024,4,1677500,2,0,...,2618.0,6,2618.0,30/09/2024,2618.0,2618.0,0.0,SERVICIO EXEQUIAL,3,D


In [13]:
print(df_staging.head())  # Para ver los primeros registros
print(df_staging.dtypes)  # Para verificar los tipos de datos


   documento_identidad nombre           apellido sexo  estado_civil  \
0               294064  ISBEL  CHAMIZO HERNANDEZ    M             4   
1               294064  ISBEL  CHAMIZO HERNANDEZ    M             4   
2               294064  ISBEL  CHAMIZO HERNANDEZ    M             4   
3               294064  ISBEL  CHAMIZO HERNANDEZ    M             4   
4               294064  ISBEL  CHAMIZO HERNANDEZ    M             4   

  fecha_ingreso  tipo_salario  salario  estrato  tipovehiculo  ...  \
0    2003-12-12             4  2600000        0             0  ...   
1    2003-12-12             4  2600000        0             0  ...   
2    2003-12-12             4  2600000        0             0  ...   
3    2003-12-12             4  2600000        0             0  ...   
4    2003-12-12             4  2600000        0             0  ...   

   vlr_credito cuopen valor_pagado  fecha_pago mora_causado mora_abono  \
0      52000.0      0      52000.0  28/02/2023      52000.0    52000.0   
1   

In [15]:
# Lista de columnas de fecha
date_columns = [
    "fecha_ingreso", "fecsolic", "fecaprob", "fecfact", "fecdesc",
    "fecultcau", "fecultpago", "fecvemto"
]

# Intentar convertir todas las columnas de fecha a datetime.date
for col in date_columns:
    if col in df_staging.columns:
        df_staging[col] = pd.to_datetime(df_staging[col], format="%d/%m/%Y", errors="coerce").dt.date

# Verificar que ahora sean del tipo correcto
print(df_staging[date_columns].dtypes)
print(df_staging[date_columns].head())

# Insertar en la base de datos
with engine.connect() as conn:
    df_staging.to_sql("saldos_staging", con=engine, if_exists="append", index=False)



#with engine.connect() as conn:
#    df_staging.to_sql("saldos_staging", con=engine, if_exists="append", index=False)

print("Los datos cargados desde archivo CSV, se almacenaron correctamente en la tabla: saldos_staging")

fecha_ingreso    object
fecsolic         object
fecaprob         object
fecfact          object
fecdesc          object
fecultcau        object
fecultpago       object
fecvemto         object
dtype: object
  fecha_ingreso    fecsolic    fecaprob     fecfact     fecdesc   fecultcau  \
0    2003-12-12  2023-02-07  2023-02-07  2023-02-07  2023-02-28  2025-02-28   
1    2003-12-12  2023-02-07  2023-02-07  2023-02-07  2023-02-28  2025-02-28   
2    2003-12-12  2023-02-07  2023-02-07  2023-02-07  2023-02-28  2025-02-28   
3    2003-12-12  2023-02-07  2023-02-07  2023-02-07  2023-02-28  2025-02-28   
4    2003-12-12  2023-02-07  2023-02-07  2023-02-07  2023-02-28  2025-02-28   

   fecultpago fecvemto  
0  2025-01-31      NaT  
1  2025-01-31      NaT  
2  2025-01-31      NaT  
3  2025-01-31      NaT  
4  2025-01-31      NaT  
Los datos cargados desde archivo CSV, se almacenaron correctamente en la tabla: saldos_staging


**Consultamos los datos de la tabla de *saldos_staging* para realizar el proceso de transformación**

In [ ]:
with engine.connect() as conn:
    db_staging_df = pd.read_sql("SELECT * FROM saldos_staging", conn)
    
db_staging_df.sample(n=15)

,id,documento_identidad,nombre,apellido,sexo,estado_civil,fecha_ingreso,tipo_salario,salario,estrato,...,cuopen,valor_pagado,fecha_pago,mora_causado,mora_abono,mora_saldo,descripcion,codahor,debcre,fecha_registro
7415,7416,6558019,SERGIO,RESTREPO RAMIREZ,M,1,2013-03-07,4,3512000.0,3,...,87.0,481522.0,30/09/2024,481522.0,481522.0,0.0,PTMO DE CONSUMO,4,D,2025-02-27 22:57:28.210469
44888,44889,16642992,ABELARDO,TELLO GARCIA,M,1,2000-01-25,2,1868400.0,0,...,1.0,NaN,None,NaN,NaN,NaN,A FAVOR,3,C,2025-02-27 22:57:28.210469
58549,58550,29939762,SANDRA,HOLGUIN DAZA,F,4,2011-04-05,4,2100000.0,0,...,3.0,NaN,None,2617.0,0.0,2617.0,SERVICIO EXEQUIAL,3,D,2025-02-27 22:57:28.210469
69409,69410,31246131,SONIA,NAVARRETE,F,1,1996-11-30,2,5443467.0,0,...,6.0,25466.0,21/06/2024,25466.0,25466.0,0.0,BON-SOL,3,D,2025-02-27 22:57:28.210469
25628,25629,16453763,NODIER,PULIDO ALVAREZ,M,4,2003-11-28,4,3927000.0,3,...,0.0,78540.0,30/04/2023,78540.0,78540.0,0.0,APORTES,1,C,2025-02-27 22:57:28.210469
30518,30518,16454682,MARCO FIDEL,SUAREZ ARCINIEGAS,M,4,1999-05-15,4,2300000.0,3,...,1.0,NaN,None,NaN,NaN,NaN,REVALORIZACION,3,C,2025-02-27 22:57:28.210469
13694,13695,16203921,MILTON,BECERRA SALDARRIAGA,M,1,2021-04-27,4,1700000.0,2,...,0.0,34156.0,31/12/2024,34000.0,34000.0,0.0,A/PERM,2,C,2025-02-27 22:57:28.210469
166989,166990,1118314105,VALERIA,PASSOS NIETO,F,1,2024-05-15,4,3900000.0,4,...,5.0,185072.0,2/08/2024,370144.0,185072.0,185072.0,PTMO DE CONSUMO,4,D,2025-02-27 22:57:28.210469
117076,117077,94364300,JUAN ISMAEL,ERAZO ORDO EZ,M,2,2007-10-25,4,4001802.0,2,...,26.0,2212.0,30/09/2023,2212.0,2212.0,0.0,SERVICIO EXEQUIAL,3,D,2025-02-27 22:57:28.210469
147383,147384,1118295643,BRIAN RICK,MU OZ RUBIO,M,4,2011-05-23,4,1980000.0,2,...,0.0,35474.0,31/10/2024,34000.0,34000.0,0.0,A/PERM,2,C,2025-02-27 22:57:28.210469


**### Hasta aqui la primera fase del proyecto ###**